In [1]:
#!pip install hmmlearn

In [1]:
import numpy as np
from preprocessEMG import train_valid_test_split, getXY
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from statistics import mean

from HMMlearn import evaluate_hmm_model, group_training_by_class, train_hmm_models_per_user, test_hmm_models_per_user
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.neural_network import MLPClassifier

from exportCSV import exportCSV

In [2]:
training, validation, testing = train_valid_test_split()

In [3]:
def evaluate_user_hmms():
    f1_macro_RMS = []
    f1_micro_RMS = []
    accuracy_RMS = []
    f1_macro = []
    f1_micro = []
    accuracy = []
    for i in range(36):
        print("at user {}".format(i))
        trainRMSX, trainX, trainY = getXY(training[i])
        validRMSX, validX, validY = getXY(validation[i])
        testRMSX, testX, testY = getXY(testing[i])
        # combine validatation and training together
        trainRMSX.extend(validRMSX)
        trainX.extend(validX)
        trainY.extend(validY)
        # classes as ints
        trainY = [int(floatclass) for floatclass in trainY]
        testY = [int(floatclass) for floatclass in testY]

        '''
        # get input in list of list format (RMS inputs are already as such)
        # currently 3D - what to do?
        trainX = np.asarray([X.values.tolist() for X in trainX])
        testX = np.asarray([X.values.tolist() for X in testX])
        # trainX.reshape()
        # testX.reshape()
        print("trainX shape: {}".format(trainX.shape))
        print("testX shape: {}".format(testX.shape))
        '''

        # RMS windows
        # train and test the model
        predictRMSY, _, _ = evaluate_hmm_model(trainRMSX, trainY, testRMSX, testY)
        # evaluate the model
        f1_macro_RMS.append(f1_score(testY, predictRMSY, average='macro'))
        f1_micro_RMS.append(f1_score(testY, predictRMSY, average='micro'))
        accuracy_RMS.append(accuracy_score(testY, predictRMSY))

        '''
        # Raw windows
        # train and test the model
        predictY, _, _ = evaluate_hmm_model(trainX, trainY, testX, testY)
        # evaluate the model 
        f1_macro.append(f1_score(testY, predictY, average='macro'))
        f1_micro.append(f1_score(testY, predictY, average='micro'))
        accuracy.append(accuracy_score(testY, predictY))
        '''
    print("RMS! Macro-F1: {}, Micro-F1: {}, Accuracy: {}".format(mean(f1_macro_RMS), mean(f1_micro_RMS), mean(accuracy_RMS)))
    #print("Macro-F1: {}, Micro-F1: {}, Accuracy: {}".format(mean(f1_macro), mean(f1_micro), mean(accuracy)))
    exportCSV(f1_macro_RMS, "hmm_f1_macro_RMS.csv")
    exportCSV(f1_micro_RMS, "hmm_f1_micro_RMS.csv")
    exportCSV(accuracy_RMS, "hmm_accuracy_RMS.csv")
    return

In [4]:
def evaluate_single_hmm():
    f1_macro_RMS = []
    f1_micro_RMS = []
    accuracy_RMS = []
    f1_macro = []
    f1_micro = []
    accuracy = []
    combineTrainX = []
    combineTrainY = []
    for i in range(36):
        # print("combining data: at user {}".format(i))
        trainRMSX, trainX, trainY = getXY(training[i])
        validRMSX, validX, validY = getXY(validation[i])
        # combine validatation and training together
        trainRMSX.extend(validRMSX)
        trainX.extend(validX)
        trainY.extend(validY)

        combineTrainX.extend(trainRMSX)
        combineTrainY.extend(trainY)
        combineTrainY = [int(floatclass) for floatclass in combineTrainY]

    # train
    # Reorder train data by class
    classes = 6
    x_train_in, _ = group_training_by_class(classes, combineTrainX, combineTrainY)
    # Make models for each class
    hmm_models = train_hmm_models_per_user(classes, x_train_in)

    # test the model for each user
    for i in range(36):
        print("testing: at user {}".format(i))
        testRMSX, testX, testY = getXY(testing[i])
        # classes as ints
        testY = [int(floatclass) for floatclass in testY]

        # RMS windows
        # test the model
        # Classify each sample
        predictRMSY = test_hmm_models_per_user(classes, testRMSX, hmm_models)
        # evaluate the model
        f1_macro_RMS.append(f1_score(testY, predictRMSY, average='macro'))
        f1_micro_RMS.append(f1_score(testY, predictRMSY, average='micro'))
        accuracy_RMS.append(accuracy_score(testY, predictRMSY))
        #print("\t RMS! Macro-F1: {}, Micro-F1: {}, Accuracy: {}".format(mean(f1_macro_RMS), mean(f1_micro_RMS), mean(accuracy_RMS)))

        '''
        # Raw windows
        # train and test the model
        predictY, _, _ = evaluate_hmm_model(trainX, trainY, testX, testY)
        # evaluate the model 
        f1_macro.append(f1_score(testY, predictY, average='macro'))
        f1_micro.append(f1_score(testY, predictY, average='micro'))
        accuracy.append(accuracy_score(testY, predictY))
        '''
    print("RMS! Macro-F1: {}, Micro-F1: {}, Accuracy: {}".format(mean(f1_macro_RMS), mean(f1_micro_RMS), mean(accuracy_RMS)))
    #print("Macro-F1: {}, Micro-F1: {}, Accuracy: {}".format(mean(f1_macro), mean(f1_micro), mean(accuracy)))
    exportCSV(f1_macro_RMS, "hmm_f1_macro_RMS.csv")
    exportCSV(f1_micro_RMS, "hmm_f1_micro_RMS.csv")
    exportCSV(accuracy_RMS, "hmm_accuracy_RMS.csv")
    return

In [5]:
evaluate_user_hmms() #GaussianHMM

at user 0
at user 1
at user 2
at user 3
at user 4
at user 5
at user 6
at user 7
at user 8
at user 9
at user 10
at user 11
at user 12
at user 13


C:\Users\Olufogorehan\PycharmProjects\vidhyaexample\venv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


at user 14
at user 15
at user 16
at user 17
at user 18
at user 19
at user 20
at user 21
at user 22
at user 23
at user 24
at user 25
at user 26
at user 27
at user 28
at user 29
at user 30
at user 31
at user 32
at user 33
at user 34
at user 35
RMS! Macro-F1: 0.8882144092111502, Micro-F1: 0.893590040089369, Accuracy: 0.893590040089369


In [8]:
evaluate_user_hmms() #GMMHMM

at user 0
at user 1
at user 2
at user 3
at user 4
at user 5
at user 6
at user 7
at user 8
at user 9
at user 10
at user 11
at user 12
at user 13


C:\Users\Olufogorehan\PycharmProjects\vidhyaexample\venv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


at user 14
at user 15
at user 16
at user 17
at user 18
at user 19
at user 20
at user 21
at user 22
at user 23
at user 24
at user 25
at user 26
at user 27
at user 28
at user 29
at user 30
at user 31
at user 32
at user 33
at user 34
at user 35
RMS! Macro-F1: 0.8762064384472914, Micro-F1: 0.8836330576324182, Accuracy: 0.8836330576324182


In [9]:
evaluate_user_hmms()

at user 0
at user 1
at user 2
at user 3
at user 4
at user 5
at user 6
at user 7
at user 8
at user 9
at user 10
at user 11
at user 12
at user 13


C:\Users\Olufogorehan\PycharmProjects\vidhyaexample\venv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


at user 14
at user 15
at user 16
at user 17
at user 18
at user 19
at user 20
at user 21
at user 22
at user 23
at user 24
at user 25
at user 26
at user 27
at user 28
at user 29
at user 30
at user 31
at user 32
at user 33
at user 34
at user 35
RMS! Macro-F1: 0.8762064384472914, Micro-F1: 0.8836330576324182, Accuracy: 0.8836330576324182


In [ ]:
evaluate_single_hmm() #GaussianHMM

In [7]:
evaluate_single_hmm() #GMMHMM

testing: at user 0
testing: at user 1
testing: at user 2
testing: at user 3
testing: at user 4
testing: at user 5


C:\Users\Olufogorehan\PycharmProjects\vidhyaexample\venv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


testing: at user 6
testing: at user 7
testing: at user 8
testing: at user 9
testing: at user 10
testing: at user 11
testing: at user 12
testing: at user 13
testing: at user 14
testing: at user 15
testing: at user 16
testing: at user 17
testing: at user 18
testing: at user 19
testing: at user 20
testing: at user 21
testing: at user 22
testing: at user 23
testing: at user 24
testing: at user 25
testing: at user 26
testing: at user 27
testing: at user 28
testing: at user 29
testing: at user 30
testing: at user 31
testing: at user 32
testing: at user 33
testing: at user 34
testing: at user 35
RMS! Macro-F1: 0.7646065455329526, Micro-F1: 0.7802822452433028, Accuracy: 0.7802822452433028
